In [1]:
!git clone https://github.com/Skuldur/Classical-Piano-Composer.git

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 156 (delta 0), reused 1 (delta 0), pack-reused 150
Receiving objects: 100% (156/156), 117.92 MiB | 10.68 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
!ls
!pwd
%cd Classical-Piano-Composer
!mkdir data2


Classical-Piano-Composer  sample_data
/content
/content/Classical-Piano-Composer


In [3]:
!wget -p ./data2 "http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz" 

--2019-02-13 11:58:33--  http://./data2
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
--2019-02-13 11:58:33--  http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
Resolving hog.ee.columbia.edu (hog.ee.columbia.edu)... 128.59.66.5
Connecting to hog.ee.columbia.edu (hog.ee.columbia.edu)|128.59.66.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234283029 (223M) [application/x-gzip]
Saving to: ‘hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz’

hog.ee.columbia.edu 100%[===================>] 223.43M  11.2MB/s    in 22s     

2019-02-13 11:58:55 (10.3 MB/s) - ‘hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz’ saved [234283029/234283029]

FINISHED --2019-02-13 11:58:55--
Total wall clock time: 22s
Downloaded: 1 files, 223M in 22s (10.3 MB/s)


In [4]:
!tar xvzf hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz -C ./data2

clean_midi/
clean_midi/Hugues Aufray/
clean_midi/Hugues Aufray/Celine.mid
clean_midi/Linda Ronstadt/
clean_midi/Linda Ronstadt/When Will I Be Loved.mid
clean_midi/Linda Ronstadt/You're No Good.mid
clean_midi/Linda Ronstadt/Different Drum.mid
clean_midi/Linda Ronstadt/Just One Look.mid
clean_midi/Linda Ronstadt/Blue Bayou.mid
clean_midi/Cabrel Francis/
clean_midi/Cabrel Francis/Repondez-moi.mid
clean_midi/Cabrel Francis/Encore et encore.mid
clean_midi/Cabrel Francis/Question d'equilibre.mid
clean_midi/Cabrel Francis/Si tu la croises un jour.mid
clean_midi/Cabrel Francis/L'encre de tes yeux.mid
clean_midi/Cabrel Francis/Quelqu'un de l'Interieur.mid
clean_midi/Salvatore Adamo/
clean_midi/Salvatore Adamo/Les Filles du bord de mer.1.mid
clean_midi/Salvatore Adamo/Une larme aux nuages.mid
clean_midi/Salvatore Adamo/Vous permettez Monsieur ?.mid
clean_midi/Salvatore Adamo/Sans toi, ma mie.mid
clean_midi/Salvatore Adamo/C'est ma vie.1.mid
clean_midi/Salvatore Adamo/Une meche de cheveux.mid
cle

In [5]:
!find ./data2/clean_midi -name '*.mid' -exec cp -t ./midi_songs {} +


cp: will not overwrite just-created './midi_songs/Tell Him.mid' with './data2/clean_midi/Celine Dion/Tell Him.mid'
cp: will not overwrite just-created './midi_songs/Ti sento.mid' with './data2/clean_midi/LIGABUE/Ti sento.mid'
cp: will not overwrite just-created './midi_songs/Just the Two of Us.mid' with './data2/clean_midi/Will Smith/Just the Two of Us.mid'
cp: will not overwrite just-created './midi_songs/Vivere.mid' with './data2/clean_midi/Rossi Vasco/Vivere.mid'
cp: will not overwrite just-created './midi_songs/Ogni volta.mid' with './data2/clean_midi/Rossi Vasco/Ogni volta.mid'
cp: will not overwrite just-created './midi_songs/Sweet Dreams.mid' with './data2/clean_midi/Patsy Cline/Sweet Dreams.mid'
cp: will not overwrite just-created './midi_songs/Kansas City.mid' with './data2/clean_midi/Domino Fats/Kansas City.mid'
cp: will not overwrite just-created "./midi_songs/Don't Be Cruel.mid" with "./data2/clean_midi/Elvis/Don't Be Cruel.mid"
cp: will not overwrite just-created './midi_s

In [6]:
%cd midi_songs

/content/Classical-Piano-Composer/midi_songs


In [9]:
!grep -Z -l 'Si' *.mid | xargs -0 rm
!grep -Z -l 'b' *.mid | xargs -0 rm
!grep -Z -l '(' *.mid | xargs -0 rm

!ls -l | wc -l


rm: missing operand
Try 'rm --help' for more information.
rm: missing operand
Try 'rm --help' for more information.
68


In [10]:
%cd ..

/content/Classical-Piano-Composer


In [12]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        try:
            midi = converter.parse(file)
        except:
            continue
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()

Parsing midi_songs/Blackbird.3.mid
Parsing midi_songs/Bleeding Me.1.mid
Parsing midi_songs/Opinion.mid
Parsing midi_songs/Laguna Sunrise.mid
Parsing midi_songs/Love Me Do.7.mid
Parsing midi_songs/Diamond Dogs.mid
Parsing midi_songs/Pennyroyal Tea.5.mid
Parsing midi_songs/Mistero.mid
Parsing midi_songs/You Wreck Me.mid
Parsing midi_songs/Boogie Man.mid
Parsing midi_songs/Si tu la croises un jour.mid
Parsing midi_songs/If Looks Could Kill.mid
Parsing midi_songs/No dottore.mid
Parsing midi_songs/Die Another Day.mid
Parsing midi_songs/Io gli ho detto no.mid
Parsing midi_songs/What It Feels Like For a Girl.mid
Parsing midi_songs/Beautiful In My Eyes.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/Who's That Girl.1.mid
Parsing midi_songs/The Things You Said.1.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/Witchy Woman.4.mid
Parsing midi_songs/Ami ancora Elisa.mid
Parsing midi_songs/Into the Mystic.mid
Parsing midi_songs/It Hurts to Be in Love.mid
Parsing midi

In [13]:
!python predict.py

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 1659, in _create_c_op
    c_op = c_api.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Dimension 1 in both shapes must be equal, but are 255 and 359. Shapes are [256,255] and [256,359]. for 'Assign_11' (op: 'Assign') with input shapes: [256,255], [256,359].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "predict.py", line 134, in <module>
    generate()
  File "predict.py", line 24, in generate
    model = create_network(normalized_input, n_vocab)
  File "predict.py", line 70, in create_network
    model.load_weights('weights.hdf5')
  File "/usr/loca